In [141]:
import logging
import select
import socket


class EthComm(object):
    def __init__(self, host, port, EOL='\r\n'):
        object.__init__(self)
        self.sock = None
        self.host = host
        self.port = port
        self.EOL = EOL

    def connectSock(self, timeout=10.):
        """| Connect socket if self.sock is None.

        :return: - socket
        """
        if self.sock is None:
            s = self.createSock()
            s.settimeout(timeout)
            s.connect((self.host, self.port))

            self.sock = s

        return self.sock

    def createSock(self):
        return socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    def closeSock(self):
        """| Close the socket.

        :raise: Exception if closing socket has failed
        """
        try:
            self.sock.close()
        except:
            pass

        self.sock = None

    def sendOneCommand(self, cmdStr, doClose=False, cmd=None):
        """| Send one command and return one response.

        :param cmdStr: (str) The command to send.
        :param doClose: If True (the default), the device socket is closed before returning.
        :param cmd: on going command
        :return: reply : the single response string, with EOLs stripped.
        :raise: IOError : from any communication errors.
        """

        if cmd is None:
            cmd = self.actor.bcast

        fullCmd = ('%s%s' % (cmdStr, self.EOL)).encode('utf-8')
        print('sending %r', fullCmd)

        s = self.connectSock()

        try:
            s.sendall(fullCmd)

        except:
            self.closeSock()
            raise

        reply = self.getOneResponse(sock=s, cmd=cmd)

        if doClose:
            self.closeSock()

        return reply

    def getOneResponse(self, sock=None, cmd=None):
        """| Attempt to receive data from the socket.

        :param sock: socket
        :param cmd: command
        :return: reply : the single response string, with EOLs stripped.
        :raise: IOError : from any communication errors.
        """
        if sock is None:
            sock = self.connectSock()

        ret = self.ioBuffer.getOneResponse(sock=sock, cmd=cmd)
        reply = ret.strip()

        print('received %r', reply)

        return reply


class BufferedSocket(object):
    """ Buffer the input from a socket and block it into lines. """

    def __init__(self, name, sock=None, loggerName=None, EOL='\n', timeout=10.,
                 logLevel=logging.INFO):
        self.EOL = EOL
        self.sock = sock
        self.name = name
        self.timeout = timeout

        self.buffer = ''

    def getOutput(self, sock=None, timeout=None, cmd=None):
        """ Block/timeout for input, then return all (<=1kB) available input. """

        if sock is None:
            sock = self.sock
        if timeout is None:
            timeout = self.timeout

        readers, writers, broken = select.select([sock.fileno()], [], [], timeout)
        if len(readers) == 0:
            cmd.warn('text="Timed out reading character from %s"' % self.name)
            raise IOError

        return sock.recv(1024).decode('utf8', 'ignore')

    def getOneResponse(self, sock=None, timeout=None, cmd=None, doRaise=False):
        """ Return the next available complete line. Fetch new input if necessary.

        Args
        ----
        sock : socket
           Uses self.sock if not set.
        timeout : float
           Uses self.timeout if not set.

        Returns
        -------
        str or None : a single line of response text, with EOL character(s) stripped.
        """
        while self.buffer.find(self.EOL) == -1:
            try:
                more = self.getOutput(sock=sock, timeout=timeout, cmd=cmd)
                if not more:
                    if doRaise:
                        raise IOError
                    else:
                        return self.getOneResponse(sock=sock, timeout=timeout, cmd=cmd, doRaise=True)

            except IOError:
                return ''

            print('%s added: %r' % (self.name, more))
            self.buffer += more

        eolAt = self.buffer.find(self.EOL)
        ret = self.buffer[:eolAt]

        self.buffer = self.buffer[eolAt + len(self.EOL):]

        return ret

class Cmd(object):
    def __init__(self):
        self.warn = print
        self.inform = print
        self.fail = print
        self.finish = print
    

In [210]:
host = 'filterwheel-dcb'
port = 9000
EOL = '\r\n'

In [292]:
cmd = Cmd()

In [293]:
sock = EthComm(host=host, port=port, EOL='\r\n')
sock.ioBuffer = BufferedSocket('socketIO', EOL='\n', timeout=5)
s = sock.connectSock()

In [166]:
wheel = 'linewheel'
position = '1'

In [167]:
ret = sock.sendOneCommand(f'{wheel} {position}', cmd=cmd)
cmd.inform(f'text="{ret}"')

while 'Moved to position' not in ret:
    ret = sock.getOneResponse(cmd=cmd)
    cmd.inform(f'text="{ret}"')

__, position = ret.split('Moved to position')
position = int(position)

sending %r b'linewheel 1\r\n'
socketIO added: 'port0 = UL  port1 = LL\n'
received %r port0 = UL  port1 = LL
text="port0 = UL  port1 = LL"
socketIO added: 'lineid = 0\n'
received %r lineid = 0
text="lineid = 0"
socketIO added: 'Setting FW 0 to position 1\nattached 2 filter wheel(s):\nindex 0: ID 0 Name EFW \nindex 1: ID 1 Name EFW \nselecting 0 \n5 slots: 1 2 3 4 5 \ncurrent position: 2\nMoving...\nMoved to position 1\n'
received %r Setting FW 0 to position 1
text="Setting FW 0 to position 1"
received %r attached 2 filter wheel(s):
text="attached 2 filter wheel(s):"
received %r index 0: ID 0 Name EFW
text="index 0: ID 0 Name EFW"
received %r index 1: ID 1 Name EFW
text="index 1: ID 1 Name EFW"
received %r selecting 0
text="selecting 0"
received %r 5 slots: 1 2 3 4 5
text="5 slots: 1 2 3 4 5"
received %r current position: 2
text="current position: 2"
received %r Moving...
text="Moving..."
received %r Moved to position 1
text="Moved to position 1"


In [299]:
ret = sock.sendOneCommand(f'adccalib ', cmd=cmd)

sending %r b'adccalib \r\n'
socketIO added: 'Turn off all lamps so that the integrating sphere is dark.\n'
received %r Turn off all lamps so that the integrating sphere is dark.


In [302]:
ret = sock.sendOneCommand(f'continue ', cmd=cmd)

sending %r b'continue \r\n'
socketIO added: 'iteration 1 z1=0.0491  z2=0.0486 \n'
received %r iteration 1 z1=0.0491  z2=0.0486


In [301]:
ret[-1]

'.'

In [274]:
cmdStr = 'adccalib '
EOL = '\r\n'

In [275]:
fullCmd = ('%s%s' % (cmdStr, EOL)).encode('latin-1')
s = sock.connectSock()

In [278]:
s.sendall(fullCmd)

In [303]:
s.recv(1024)

b'iteration 2 z1=0.0491  z2=0.0488 \niteration 3 z1=0.0491  z2=0.0486 \niteration 4 z1=0.0491  z2=0.0486 \niteration 5 z1=0.0491  z2=0.0486 \n\nZeros for channel 1, 2 = .0491, .0486\n'

In [284]:
sock.closeSock()

In [ ]:
s.sendall('toto'.encode('latin-1'))

In [172]:
print('port0 = |UL|  port1 = |LL| \nCalibrating FW 1 \nattached 2 filter wheel(s): \nindex 0: ID 0 Name EFW \nindex 1: ID 1 Name EFW \nselecting 1\nCalibrating \nDone\n')

port0 = |UL|  port1 = |LL| 
Calibrating FW 1 
attached 2 filter wheel(s): 
index 0: ID 0 Name EFW 
index 1: ID 1 Name EFW 
selecting 1
Calibrating 
Done



In [209]:
s.close()

In [ ]:
print(s)

In [71]:
ret.split('\r')

['Bonn Shutter  Vers: Dec 15 2014@17:21:49',
 '\n----------------------------------------',
 '\n\nsh           show all profile parameters (verbose mode)',
 '\nma x pos     move with freq. x to abs. position pos ',
 '\nmr x stp     move with freq. x by stp steps ',
 '\nmq        \t  stop both motors (ma, mr only)',
 '\nsp           motor position to ser. port',
 '\nbs x         set blade start position (40=1mm) ',
 '\nbd x         set blade travel distance (40=1mm) ',
 '\nac x         set acceleration parameter (1,2,3,4...)',
 '\nvm x         set maximum velocity (steps/sec)',
 '\nth x         set motor/encoder mismatch threshold to x steps',
 '\nls x         set position reset speed',
 '\nlt x         set position reset timeout (msec, <32000)',
 '\nfd           reset to factory defaults',
 "\nsu x         suppress error (x = 1) or don't (x = 0)",
 '\npp           Get all 7 profile parameters',
 '\nsb x         send status byte x (1-2 Comm; 3-4 A; 5-6 B)',
 '\nos           open shutter